In [2]:
from pathlib import Path

import pyvista as pv
import pyacvd
import nibabel as nib
import numpy as np


pv.set_jupyter_backend('html')

ssm_nii_dir = Path.cwd().parent / "output_ssm_nii"
vtk_dir = Path.cwd().parent / "output_ssm_vtk"

In [ ]:
for ssm_case in ssm_nii_dir.iterdir():
    if not ssm_case.is_dir():
        continue
    label_dir = ssm_case / "label"
    for label_nii_file in label_dir.glob("*.nii.gz"):
        label_nii = nib.load(str(label_nii_file))
        label_data = label_nii.get_fdata()
        label_ids = sorted(np.unique(label_data).astype(np.int8))[1:]
        surface_all = pv.PolyData()
        vtk_file = vtk_dir / label_nii_file.relative_to(ssm_nii_dir).with_suffix('').with_suffix(".vtk")
        print(vtk_file)
        for label_id in label_ids:
            assert label_id > 0
            surface = pv.wrap((label_data == label_id).astype(np.uint8)).contour([1], method="flying_edges").triangulate()
            clus = pyacvd.Clustering(surface)
            clus.subdivide(3)
            clus.cluster(1000)
            surface = clus.create_mesh()
            surface.point_data["label"] = np.ones(surface.n_points) * label_id
            surface_all = surface_all.append_polydata(surface)
        
        surface_all.save(str(vtk_file))


            


/media/data3/sj/Data/Phatom/output_ssm_vtk/male_pt184/label/ssm_000.vtk


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

/media/data3/sj/Data/Phatom/output_ssm_vtk/male_pt184/label/ssm_003.vtk


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

/media/data3/sj/Data/Phatom/output_ssm_vtk/male_pt184/label/ssm_004.vtk
